In [1]:
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
from keras.models import load_model
import matplotlib.pyplot as plt
import matplotlib
import cv2

2021-08-13 10:40:25.012093: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '2'
# print(device_lib.list_local_devices())

In [3]:
# os.chdir('/home/aijjeh/Desktop/Phd_Project/Full_wavefield_frames_time_series_project/Datasets/label_set')
# model_name = 'oxford_segmentation_64.h5'

os.chdir('/home/aijjeh/Desktop/Phd_Project/Sequence_to_sequence/h5_models/')
model_name = "UNet_Xception_style_model_24.h5"

model = load_model(model_name, compile=False)
# model.summary()

2021-08-13 10:40:27.040866: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-13 10:40:27.042178: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-08-13 10:40:27.083920: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-08-13 10:40:27.083964: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-08-13 10:40:27.085938: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-08-13 10:40:27.086000: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [5]:
input_dir = "/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset2_bottom_out/"
target_dir = "/home/aijjeh/Desktop/Phd_Project/test_labels/"
my_list = os.listdir(input_dir)
my_list.sort(key=natural_keys)
print(len(my_list))

475


In [6]:
height = 512
width =  512
img_size = (height, width)
num_classes = 2
batch_size = 1
time_stamps =  24
input_img_paths_total = [None]*95

In [7]:
for i in range(95):
    input_dir = "/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset2_bottom_out/" +'%d_output' % (i+381) 
    input_img_paths = sorted( 
        [
            os.path.join(input_dir, fname) 
            for fname in os.listdir(input_dir)    
            if fname.endswith(".png")
        ] 
    )
    input_img_paths.sort(key=natural_keys)
    input_img_paths_total[i] = input_img_paths
print(len(input_img_paths_total))

95


In [8]:
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)
target_img_paths.sort(key=natural_keys)
print(len(target_img_paths))

95


In [9]:
os.chdir('/home/aijjeh/Desktop/Phd_Project/Full_wavefield_frames_time_series_project/Datasets/label_set')
file_frame = np.load('frames_initial.npy')
file_frame = file_frame[380:]
print(len(file_frame))

95


In [10]:
class Full_wavefield_frames(keras.utils.Sequence):
    def __init__(self, batch_size, img_size, input_img_paths_total_, target_img_paths_, time_stamps):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths_total = input_img_paths_total_
        self.target_img_paths = target_img_paths_
        self.time_stamps = time_stamps
    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size        
        batch_input_img_paths = self.input_img_paths_total[i:i+self.batch_size]      
        batch_target_img_paths = self.target_img_paths[i:i+self.batch_size] 
        
        x = np.zeros((self.batch_size,) + (self.time_stamps,) + self.img_size + (1,), dtype="float16") #  
        
        for batch_num in range(self.batch_size):
            batch_input_img_paths = batch_input_img_paths[batch_num][file_frame[i]:file_frame[i]+self.time_stamps] 
            for j, path in enumerate(batch_input_img_paths):
                img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
                img = np.expand_dims(img,2)
                img = img / 255.0
                x[batch_num][j] = img
        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float16")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")            
            img = np.expand_dims(img, 2)
            img = img / 255.0
            y[j] = img
                # Ground truth labels are 1, 2, 3. Subtract one to make them 0, 1, 2:
                # y[j] -= 1
        return x, y

In [11]:
test_input_img_paths = input_img_paths_total[:]
test_target_img_paths = target_img_paths[:]

In [12]:
val_gen = Full_wavefield_frames(batch_size, img_size, input_img_paths_total, target_img_paths, time_stamps)

In [13]:
val_preds = model.predict(val_gen)

2021-08-13 10:40:33.720027: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-13 10:40:33.739083: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2021-08-13 10:40:35.140590: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-08-13 10:40:36.844901: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


# Load cmap color

In [14]:
path_to_csv = '/home/aijjeh/aijjeh_rexio_share/PhD/cmap_flipped_jet256.csv'
cmap = matplotlib.colors.ListedColormap(["blue", "green", "red"], name=path_to_csv, N=None)

In [15]:
def display_mask(i):
    """Quick utility to display a model's prediction."""
    # mask = np.argmax(val_preds[i], axis=-1)
    mask = val_preds[i]
    # mask = np.expand_dims(mask, axis=-1)
    img = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    plt.imshow(img)
    plt.show()
    # display(img)

In [16]:
def calc_IoU(predicted_image, truth_img, img_num):
    predicted_image = np.asarray(predicted_image) # 
    predicted_image = predicted_image.astype('float64')
    ret, predicted_image = cv2.threshold(predicted_image, .5, 1.0, cv2.THRESH_BINARY)
    truth_img = np.asarray(truth_img)# 
    truth_img = truth_img.astype('float64')
    truth_img = truth_img[:,:,0]
    InterSectionArray = cv2.bitwise_and(predicted_image, truth_img)
    UnionArray = cv2.bitwise_or(predicted_image, truth_img)
    I1 = np.count_nonzero(InterSectionArray)
    U = np.count_nonzero(UnionArray)
    IoU = I1 / U
    print(img_num+381,' : ',IoU)
    # gc.collect()
    return IoU

In [17]:
total_IoU  = 0

In [18]:
os.chdir('/home/aijjeh/Desktop/Phd_Project/Full_wavefield_frames_time_series_project/AE_numerical_results_24_frames_512_512/')

In [19]:
for j in range(len(val_preds)):
    ############################################################################################################
    plt.figure(figsize=(5 / 2.54, 5 / 2.54), dpi=600)
    plt.gca().set_axis_off()
    plt.axis('off')
    plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    ############################################################################################################
# Display input image
    # img_input_ = val_gen[j][0][0]
    # plt.imshow((img_input_[15]*255).astype(np.uint8), cmap='Greys')
    # plt.show()

# Display ground-truth target mask
    # os.chdir('/home/aijjeh/Desktop/Phd_Project/Full_wavefield_frames_time_series_project/AE_numerical_GT/')
    img_GT_ = PIL.ImageOps.autocontrast(load_img(test_target_img_paths[j], target_size= img_size))
    # plt.imshow(img_GT_, cmap='Greys')
    # plt.savefig('numberical_GT_%d' % (j+381))
    # plt.show()

# Display mask predicted by our model
    # display_mask(j)  
    mask = val_preds[j]
    img_pred_ = PIL.ImageOps.autocontrast(keras.preprocessing.image.array_to_img(mask))
    plt.imshow(mask, cmap=cmap)
    plt.savefig('predicted_%d' % (j + 381))
    plt.close('all')
    total_IoU = total_IoU + calc_IoU(mask, img_GT_, j)


381  :  0.9374233128834356
382  :  0.9544159544159544
383  :  0.9031007751937985
384  :  0.9090909090909091
385  :  0.9622641509433962
386  :  0.857566765578635
387  :  0.9557522123893806
388  :  0.9307282415630551
389  :  0.9553752535496958
390  :  0.7602523659305994
391  :  0.8664383561643836
392  :  0.9690576652601969
393  :  0.9731258840169731
394  :  0.9209726443768997
395  :  0.9746835443037974
396  :  0.963013698630137
397  :  0.9370748299319728
398  :  0.5
399  :  0.9512195121951219
400  :  0.924187725631769
401  :  0.03206997084548105
402  :  0.9016393442622951
403  :  0.918429003021148
404  :  0.9806451612903225
405  :  0.8215859030837004
406  :  0.9072847682119205
407  :  0.961439588688946
408  :  0.7638190954773869
409  :  0.9716981132075472
410  :  0.945273631840796
411  :  0.9454022988505747
412  :  0.9400630914826499
413  :  0.9898348157560356
414  :  0.9718076285240465
415  :  0.8706293706293706
416  :  0.9565217391304348
417  :  0.78419452887538
418  :  0.8241965973534

In [20]:
miou = total_IoU / len(val_preds)

In [21]:
print(miou)

0.8719347769144274
